In [ ]:
import os
from dotenv import load_dotenv
from moviepy.editor import VideoFileClip, concatenate_videoclips, TextClip, ImageClip, CompositeVideoClip, AudioFileClip
import pyttsx3 as tts
import whisper
import requests
from elevenlabs.client import ElevenLabs
from elevenlabs import generate, save
import google.generativeai as genai
load_dotenv()

client = ElevenLabs(api_key=os.getenv('ELEVENLABS_API_KEY'))
gemini_key = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=gemini_key)

In [ ]:
def generate_script(prompt, stories):
    model = genai.GenerativeModel('gemini-pro')
    default_prompt = """from now on return the output as regular text and the story should look
                    like it being narrated by someone, For an Example : There was once a boy fishing in the ocean
                    and he caught a fish, the fish was very big he was so happy, 
                    You have to write maximum 1000 characters I want a plain text no symbols just the script, 
                    also the script content must be in 50 seconds duration, dont include any symbols or special characters,"""
    for i in range(stories):
        story = model.generate_content(f"{default_prompt},{prompt}")
        with open(f'stories/story_{i}.txt', 'w') as f:
            f.write(story.text)

In [ ]:
def get_images(text):
    BASE = 'https://api.unsplash.com/photos/'
    PAYLOAD = {
        'query': text,
        'client_id': os.getenv('UNSPLASH_ACCESS_KEY')
    }
    response = requests.get(BASE, params=PAYLOAD)
    data = response.json()
    return data

In [ ]:
prompt = "Scary Mystery Puzzle"
stories = 5
generate_script(prompt, stories)

In [ ]:
# def speak(text):
#     audio = generate(text,
#                      voice='Rachel',
#                      model = "eleven_multilingual_v2")
#     save(audio, 'samp.mp3')
    
def speak(text, file_name):
    engine = tts.init()
    engine.setProperty('rate', 150)
    engine.save_to_file(text, f'audio/{file_name}.mp3')
    engine.runAndWait()

for i in range(stories):
    with open(f'stories/story_{i}.txt', 'r') as f:
        text = f.readlines()
        speak(text, f'story_{i}')

In [ ]:
def speech_to_text(audio_file):
    model = whisper.load_model("base")
    data = model.transcribe(audio_file, word_timestamps=True)

    start = [data['segments'][i]['words'][j]['start'] for i in range(len(data['segments'])) for j in range(len(data['segments'][i]['words']))]
    end = [data['segments'][i]['words'][j]['end'] for i in range(len(data['segments'])) for j in range(len(data['segments'][i]['words']))]
    text = [data['segments'][i]['words'][j]['word'] for i in range(len(data['segments'])) for j in range(len(data['segments'][i]['words']))]

    return [start, end, text]

In [ ]:
from moviepy.video.tools.subtitles import SubtitlesClip
from moviepy.editor import CompositeAudioClip, concatenate_audioclips

def generate_video():
    for i in range(stories):
        with open(f'stories/story_{i}.txt', 'r') as file:
            print(f'Generating video for story {i}')
            text = file.read()
        
        words = text.split(' ')

        generator = lambda txt: TextClip(txt, 
                                    fontsize=70, 
                                    color='white', 
                                    bg_color='none', 
                                    font='Arial-Bold',
                                    method = 'caption',
                                    size = (1920, 1080))

        # def clip_generator():
        #     for _ in words:
        #         yield CompositeVideoClip([ImageClip('Z_1.jpg').set_duration(0.5).set_position('center')], size=(720, 1280))

        # clips = concatenate_videoclips(clip_generator(), method="compose")

        speech = speech_to_text(f'audio/story_{i}.mp3')
        print(f'Accessing speech for story {i}')
        subs = [((speech[0][k], speech[1][k]), speech[2][k]) for k in range(len(speech[0]))]

        subtitles = SubtitlesClip(subs, generator)
        image = ImageClip('Z_1.jpg').set_duration(10).set_position('center')
        final_complete = CompositeVideoClip([image, subtitles.set_position(('center', 'center'))])
        aud = f'audio/story_{i}.mp3'
        final_complete = final_complete.set_audio(aud.set_duration(final_complete))
        
        final_complete.write_videofile(f"output_{i}.mp4", audio = aud , fps=24, bitrate = '1000', preset = 'fast', codec = 'h264_nvenc')

In [ ]:
generate_video()

In [ ]:
for l in range(len(speech[0])):
    print(((speech[0][l], speech[1][l]), speech[2][l]))